In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import scipy.io
all_labels = scipy.io.loadmat('/content/drive/My Drive/Iot Assign 1/imagelabels.mat')['labels'][0] - 1  


In [18]:
import numpy as np
np.unique(all_labels)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101], dtype=uint8)

In [19]:
len(all_labels)

8189

In [0]:
import pandas as pd
new=pd.read_csv('/content/drive/My Drive/Iot Assign 1/new_file.csv', header=None)

In [17]:
new.shape

(8189, 900)

In [20]:
import numpy as np
import tensorflow as tf

#Change USERNAME by the username of your machine
## Windows USER

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(new, all_labels, test_size=0.2, random_state=42)
y_train  = y_train.astype(int)
y_test  = y_test.astype(int)
batch_size =len(X_train)

print(X_train.shape, y_train.shape,y_test.shape )
## resclae
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# Train
X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))
# test
X_test_scaled = scaler.fit_transform(X_test.astype(np.float64))
feature_columns = [tf.feature_column.numeric_column('x', shape=X_train_scaled.shape[1:])]
X_train_scaled.shape[1:]

(6551, 900) (6551,) (1638,)


(900,)

In [0]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 30, 30, 1])

  # Convolutional Layer
  conv1 = tf.compat.v1.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer
  pool1 = tf.compat.v1.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer
  conv2 = tf.compat.v1.layers.conv2d(
      inputs=pool1,
      filters=36,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.compat.v1.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 36])
  dense = tf.compat.v1.layers.dense(inputs=pool2_flat, units=7 * 7 * 36, activation=tf.nn.relu)
  dropout = tf.compat.v1.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.compat.v1.layers.dense(inputs=dropout, units=102)
  print(tf.argmax(input=logits,axis=1))
  print(logits)
  print(labels)
  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss
  loss = tf.compat.v1.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.compat.v1.estimator.ModeKeys.TRAIN:
    #optimizer = tf.optimizers.SGD (learning_rate=0.001)
    optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.compat.v1.train.get_global_step())
    return tf.compat.v1.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics Evaluation mode
  eval_metric_ops = {
      "accuracy": tf.compat.v1.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.compat.v1.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [22]:
# Create the Estimator
flower_classifier = tf.compat.v1.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="train/mnist_convnet_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'train/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.compat.v1.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

In [0]:
train_input = tf.compat.v1.estimator.inputs.numpy_input_fn(
    x={"x": X_train_scaled},
    y=y_train,
    batch_size=16,
    num_epochs=None,
    shuffle=True)

In [29]:
flower_classifier.train(
    input_fn=train_input,
    steps=160000,
    hooks=[logging_hook])

Streaming output truncated to the last 5000 lines.
 [0.         0.         0.00000017 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.00000001 0.00000001]
 [0.0000139  0.000012   0.00016171 ... 0.00000377 0.00000913 0.00000331]] (0.355 sec)
INFO:tensorflow:global_step/sec: 141.826
INFO:tensorflow:probabilities = [[0.00000372 0.00000589 0.00004358 ... 0.00000248 0.0000079  0.00000357]
 [0.00000012 0.00000006 0.00000001 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.00000003]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.00002127 0.00004098 0.00011194 ... 0.00001033 0.00002621 0.00000518]
 [0.00000506 0.0000207  0.00009997 ... 0.00000343 0.00000596 0.00000703]] (0.351 sec)
INFO:tensorflow:loss = 1.102811336517334, step = 128800 (0.705 sec)
INFO:tensorflow:probabilities = [[0.00055707 0.00000151 0.00000344 ... 0.00000003 0.         0.00000047]
 [0.00006891 0.

In [30]:
type(X_train_scaled)

numpy.ndarray

In [31]:
# Evaluate the model and print results
eval_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    x={"x": X_test_scaled},
    y=y_test,
    num_epochs=1,
    shuffle=False)
eval_results = flower_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
Tensor("ArgMax:0", shape=(None,), dtype=int64)
Tensor("dense_1/BiasAdd:0", shape=(None, 102), dtype=float64)
Tensor("fifo_queue_DequeueUpTo:2", shape=(None,), dtype=int64, device=/device:CPU:0)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-06-13T16:39:18Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train/mnist_convnet_model/model.ckpt-160000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.36977s
INFO:tensorflow:Finished evaluation at 2020-06-13-16:39:19
INFO:tensorflow:Saving dict for global step 160000: accuracy = 0.13431014, global_step = 160000, loss = 5.79071
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 160000: train/mnist_convnet_model/model.ckpt-160000
{'accuracy': 0.13431014, 'loss': 5.79071, 'global_step': 160000}
